# **灯光捕捉（TracktheLight)**

## **Demo 视频**

摄像头根据颜色识别捕捉灯棒的移动，当灯棒从摄像头视野范围内消失后通过 Adapter 向 Scratch 发送消息，启动玫瑰窗的转动。

In [4]:
import IPython.display as ipd

ipd.Video('video/demo3_tracktheLight.mp4', width=800, height=600)

</br>

## **Demo 代码**

**视频中的示例项目主要包括两部分：**

1. 使用 OpenCV 通过颜色识别追踪灯棒（树莓派驱动的 ws281x 8X8 灯阵）

   感谢作者 Adrian Rosebroc！机器视觉部分的代码参考这篇[教程](https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/)

2. 灯棒移出摄像头捕捉范围时通过 Adapter EIM 触发 Scratch 项目中的玫瑰窗转动，项目[在此](https://create.codelab.club/projects/13195/)



In [5]:
from collections import deque
from imutils.video import VideoStream         # https://github.com/jrosebr1/imutils
import numpy as np
import imutils
import time
import cv2
from codelab_adapter_client import AdapterNode


# Adapter EIM Node 初始化
class MyNode(AdapterNode):
    NODE_ID = "eim/rose_window"    # 这个 ID 名（即 "eim/"后面的部分自己定义，配套 Scratch 项目中使用的接收和发送 EIM 消息的积木内要写与此一致的名字

    def __init__(self):
        super().__init__()
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)


node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)


# 因为是通过颜色追踪物体，这里需要确定被识别颜色的范围（HSV），H 0-179， S 和 V 0-255
redLower = (0, 0, 179)
redUpper = (179, 255, 255)

#定义用来存放被追踪物体轨迹位置的列表最长多少
pts = deque(maxlen=128) 

# 启动摄像头
vs = VideoStream(src=0).start()
time.sleep(2.0)


while True:

    frame = vs.read()
    frame = cv2.flip( frame, 1 ) 
    frame = imutils.resize(frame, width=800)
    blurred = cv2.GaussianBlur(frame,(11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)   
    mask = cv2.inRange(hsv, redLower, redUpper)
    # 关于 erode 和 dilate，参看文档解释
    # erode 磨损前景（白色光亮区域）的轮廓部分，黑暗背景变大
    # dilate 相反使前景光亮区域更大
    # 通常都是先 erode，去掉前景周围白色微小 noise，然后 dilate 使前景损失掉的再补回来
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
    # 作者解释说这里是为了去掉 mask 中那些小的光点
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)       
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
        
        
    if len(cnts)>0:
        c = max(cnts, key=cv2.contourArea)
        ((x,y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

            
            
    pts.appendleft(center)
        
    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue           
        thickness = int(np.sqrt(64/ float(i + 1))* 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
        
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
        
    if key == ord("q"):
        break
        vs.stop()
        cv2.destroyAllWindows()
                                    
                                    
    if len(cnts)==0:
        node.send_data("go")
        vs.stop()
        cv2.destroyAllWindows()
        break
                                  


</br>

## **附：确定要识别追踪物体的颜色范围（color range）**

如上面 Demo 所示，如果使用颜色识别追踪物体，需要确定物体的颜色范围，运行下方代码，就可以非常方便的拿到 HSV 取值范围。**感谢作者 Praveen！具体见这篇[文章](https://medium.com/programming-fever/how-to-find-hsv-range-of-an-object-for-computer-vision-applications-254a8eb039fc)**

In [ ]:
#finding hsv range of target object(pen)
import cv2
import numpy as np
import time

# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Initializing the webcam feed.
# 将 VideoCapture 的第 3 个（从 0 算起）特性（共 0-18 19 个特性），即 frame_width，设为 1280
# 同上，将 frame_height 设为 720
# 关于 cap.set()和 cap.get() 
# 参看文档：https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html#videocapture-get
cap = cv2.VideoCapture(0)
cap.set(3,1280)   
cap.set(4,720)    

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 trackbars that will control the lower and upper range of 
# H,S and V channels. The Arguments are like this: Name of trackbar, 
# window name, range,callback function. For Hue the range is 0-179 and
# for S,V its 0-255.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
while True:
    
    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    if not ret:
        break
        
    # Flip the frame horizontally (Not required)
    frame = cv2.flip( frame, 1 ) 
    
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time as the user changes 
    # them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # Set the lower and upper HSV range according to the value selected
    # by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask, where white represents 
    # your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # You can also visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Converting the binary mask to 3 channel image, this is just so 
    # we can stack it with the others
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack the mask, orginal frame and the filtered result
    stacked = np.hstack((mask_3,frame,res))
    
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    # If the user presses `s` then print this array.
    if key == ord('s'):
        
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as penval.npy
        np.save('hsv_value',thearray)
        break
    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()